In [ ]:
import numpy as np 
import pandas as pd 
import os
import itertools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D, Conv2DTranspose
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
from keras.optimizers import SGD
from keras.layers.merge import concatenate
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import load_img

from tensorflow.python.keras.callbacks import TensorBoard

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from model_00 import get_model
import pickle

import keras.backend as K
import tensorflow as tf

import utils
import imageio
import augment

np.random.seed(0)

In [ ]:
d_train = "/home/matthias/projects/bio-unet/data/512/train"
d_test = "/home/matthias/projects/bio-unet/data/512/test"

In [ ]:
files = utils.get_files(d_test, verbose=False)
print(files[0:10])

In [ ]:
x = imageio.imread("/home/matthias/projects/bio-unet/data/512/test/11_x.png")
y = imageio.imread("/home/matthias/projects/bio-unet/data/512/test/11_y.png")
y = y.reshape((512, 512, 1))
utils.show_img(x, y.reshape(y.shape[:-1]))
for i in range(7):
    print(i)
    xn = augment_imgarr(x, i)
    #yn = augment_imgarr(y, i)
    utils.show_img(x, xn)

In [ ]:
batches = 20
dg = dataGenerator(files,batches=batches, normalize=True, augment=True)
X, Y = next(dg)
X, Y = next(dg)
Xe, Ye = None, None
for j in range(batches):
    x = X[j][:]
    y = Y[j][:]
    y = y.reshape((512, 512))
    z = y > 0
    x[z] = 0
    utils.show_img(x, y)

In [ ]:
y = Y[0]
x = X[0]
z = y > 0
z = z.reshape((512, 512))
print(z.shape)
print(x.shape)
x[z] = 1

In [ ]:
Ye

In [ ]:
model = get_model(n_kernels=1, img_height=2084, img_width=2084)
print(model.summary())

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.98, nesterov=True),
              metrics=['accuracy'])
#K.get_session().run(tf.global_variables_initializer())
#tensorboard = TensorBoard(log_dir="logs") 
callbacks = [
             EarlyStopping(monitor='val_loss',
                           patience=2,
                           verbose=1),
             ModelCheckpoint("model01.h5",
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)]
train_generator = dataGenerator(files_train, batches=10, weights=True)
valid_generator = dataGenerator(files_valid, batches=3, weights=True)

In [ ]:
# batch_size determines the number of samples in each mini batch
# steps_per_epoch the number of batch iterations before a training epoch is considered finished.
# validation_steps
hist = model.fit_generator(generator=train_generator,
                    steps_per_epoch=1,
                    epochs=10,
                    verbose=2,
                    callbacks=callbacks,
                    validation_data=valid_generator,
                    validation_steps=1,
                    max_queue_size=2)

In [ ]:
training_loss = hist.history['loss']
test_loss = hist.history['val_loss']

epoch_count = range(1, len(training_loss) + 1)

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
with open(os.path.join(splitdata.d_data, splitdata.f_test), "rb") as file:
    files_test = pickle.load(file)
if not files_test:
    raise Exception("Could not load training files!")

In [ ]:
X_test, Y_test = preprocess.load_data(files_test)

In [ ]:
X_test = X_test / 255

In [ ]:
Y_pred = model.predict(X_test, batch_size=len(X_test), verbose=1)

In [ ]:
print(Y_pred[0])

In [ ]:
Y_pred = np.around(Y_pred)

In [ ]:
for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(Y_test[i], Y_pred[i])

In [ ]:
for i in range(Y_pred.shape[0]):
    print(np.sum(Y_pred[i]))